# Environment

## 匯入server.py


In [30]:
# 📒 job_guardian_test.ipynb

# ------------------------------------------------------------
# 測試 0: 匯入 server.py
# ------------------------------------------------------------
import importlib.util
import sys

spec = importlib.util.spec_from_file_location("server", "server.py")
server = importlib.util.module_from_spec(spec)
sys.modules["server"] = server
spec.loader.exec_module(server)

print("✅ server.py 匯入成功，MCP server 已載入。")


✅ server.py 匯入成功，MCP server 已載入。


In [11]:
# 用哪個PYTHON
import sys, platform, socket
print(sys.executable)
print(platform.system())
print(socket.getaddrinfo("mopsfin.twse.com.tw", 443))


/home/de-jung/projects/job-guardian/.venv/bin/python
Linux
[(<AddressFamily.AF_INET: 2>, <SocketKind.SOCK_STREAM: 1>, 6, '', ('117.56.218.140', 443)), (<AddressFamily.AF_INET: 2>, <SocketKind.SOCK_DGRAM: 2>, 17, '', ('117.56.218.140', 443)), (<AddressFamily.AF_INET: 2>, <SocketKind.SOCK_RAW: 3>, 0, '', ('117.56.218.140', 443)), (<AddressFamily.AF_INET: 2>, <SocketKind.SOCK_STREAM: 1>, 6, '', ('163.29.17.140', 443)), (<AddressFamily.AF_INET: 2>, <SocketKind.SOCK_DGRAM: 2>, 17, '', ('163.29.17.140', 443)), (<AddressFamily.AF_INET: 2>, <SocketKind.SOCK_RAW: 3>, 0, '', ('163.29.17.140', 443)), (<AddressFamily.AF_INET: 2>, <SocketKind.SOCK_STREAM: 1>, 6, '', ('122.147.34.140', 443)), (<AddressFamily.AF_INET: 2>, <SocketKind.SOCK_DGRAM: 2>, 17, '', ('122.147.34.140', 443)), (<AddressFamily.AF_INET: 2>, <SocketKind.SOCK_RAW: 3>, 0, '', ('122.147.34.140', 443))]


# 測試爬蟲


In [ ]:
# check certification
import sys, ssl, requests, certifi
print("Python:", sys.executable)
print("OpenSSL:", ssl.OPENSSL_VERSION)
print("Requests:", requests.__version__)
print("Certifi:", certifi.where())


In [ ]:
rows = server._fetch_csv_rows(server.ESG_URL)
print("筆數:", len(rows))
print("前5筆:", rows[0])


In [31]:
# ------------------------------------------------------------
# 測試 1: 確認爬蟲來源可下載並解析
# ------------------------------------------------------------
for url in [server.ESG_URL, server.LAB_VIO_URL, server.GE_VIO_URL]:
    rows = server._fetch_csv_rows(url)
    print(f"來源: {url}")
    print(f"共 {len(rows)} 筆，前 1 筆：")
    print(rows[0] if rows else "⚠️ 無資料")
    print("="*80)


來源: https://mopsfin.twse.com.tw/opendata/t187ap46_O_5.csv
共 850 筆，前 1 筆：
{'\ufeff出表日期': '1140925', '報告年度': '113', '公司代號': '1240', '公司名稱': '茂生農經', '員工福利平均數(仟元/人)': '982', '員工薪資平均數(仟元/人)': '841', '非擔任主管職務之全時員工薪資平均數(仟元/人)': '740', '非擔任主管之全時員工薪資中位數(仟元/人)': '649', '管理職女性主管佔比': '6.00%'}
來源: https://apiservice.mol.gov.tw/OdService/download/A17000000J-030225-svj
共 68411 筆，前 1 筆：
{'主管機關': '南投縣', '公告日期': '20250829', '處分日期': '20250829', '處分字號': '府授勞資字第 1140202257號', '事業單位名稱或負責人': '賴慶鴻即萊緹蛋糕坊', '違法法規法條': '勞動基準法第24條', '違反法規內容': '延長工作時間未依規定加給工資', '罰鍰金額': '20000', '備註說明': ''}
來源: https://apiservice.mol.gov.tw/OdService/download/A17000000J-030226-sop
共 1064 筆，前 1 筆：
{'主管機關': '新竹縣', '公告日期': '20160105', '處分日期': '20151106', '處分字號': '104年府勞資字第1040137166號', '事業單位名稱或負責人': '千才科技股份有限公司(\n高嵩岳)', '違法法規法條': '性別工作平等法第7條', '違反法規內容': '招募特定性別', '罰鍰金額': '', '備註說明': ''}


# 測試工具內函式

In [32]:
# ------------------------------------------------------------
# 測試 2: 呼叫 esg_hr 工具
# ------------------------------------------------------------
result = server.esg_hr("大甲", year=113, limit=3)
print("esg_hr 測試輸出：")
print(result)

assert isinstance(result, dict)
assert "items" in result
assert "count" in result
assert "source_url" in result
assert "meta" in result
print("✅ esg_hr output 格式正確")


esg_hr 測試輸出：
{'items': [{'公司代號': '2221', '公司名稱': '大甲', '年度': '113', '員工薪資中位數': '499', '員工薪資平均數': '563', '女性主管比例': '27.03%', '資料列原始': {'\ufeff出表日期': '1140925', '報告年度': '113', '公司代號': '2221', '公司名稱': '大甲', '員工福利平均數(仟元/人)': '696', '員工薪資平均數(仟元/人)': '563', '非擔任主管職務之全時員工薪資平均數(仟元/人)': '573', '非擔任主管之全時員工薪資中位數(仟元/人)': '499', '管理職女性主管佔比': '27.03%'}}], 'count': 1, 'source_url': 'https://mopsfin.twse.com.tw/opendata/t187ap46_O_5.csv', 'fetched_at': '2025-09-25T06:50:10Z', 'meta': {'query': '大甲', 'year': 113, 'partial_match': False}}
✅ esg_hr output 格式正確


In [17]:
# ------------------------------------------------------------
# 測試 3: 呼叫 labor_violations 工具
# ------------------------------------------------------------
result = server.labor_violations("道明修女會", since_year=2025, limit=3)
print("labor_violations 測試輸出：")
print(result)

assert isinstance(result, dict)
assert "items" in result
assert "stats" in result
assert "source_url" in result
assert "meta" in result
print("✅ labor_violations output 格式正確")


labor_violations 測試輸出：
{'items': [{'事業單位名稱': '財團法人天主教中華道明修女會附設雲林縣私立西光幼兒園(徐嘉雯)', '公告日期': '20250903', '裁處機關': '雲林縣', '違反法條': '勞動基準法第30條第6項', '違反法條內容': '出勤紀錄，未逐日記載勞工出勤情形至分鐘為止', '罰鍰金額': '20000', '資料列原始': {'主管機關': '雲林縣', '公告日期': '20250903', '處分日期': '20250826', '處分字號': '114年府勞動一字第1143425678號', '事業單位名稱或負責人': '財團法人天主教中華道明修女會附設雲林縣私立西光幼兒園(徐嘉雯)', '違法法規法條': '勞動基準法第30條第6項', '違反法規內容': '出勤紀錄，未逐日記載勞工出勤情形至分鐘為止', '罰鍰金額': '20000', '備註說明': ''}}], 'count': 1, 'stats': {'count_by_year': {'2025': 1}}, 'source_url': 'https://apiservice.mol.gov.tw/OdService/download/A17000000J-030225-svj', 'fetched_at': '2025-09-24T13:49:33Z', 'meta': {'query': '道明修女會', 'since_year': 2025, 'partial_match': True}}
✅ labor_violations output 格式正確


In [26]:
# ------------------------------------------------------------
# 測試 4: 呼叫 ge_work_equality_violations 工具
# ------------------------------------------------------------
result = server.ge_work_equality_violations("大慶", since_year=2020, limit=3)
print("ge_work_equality_violations 測試輸出：")
print(result)

assert isinstance(result, dict)
assert "items" in result
assert "stats" in result
assert "source_url" in result
assert "meta" in result
print("✅ ge_work_equality_violations output 格式正確")


ge_work_equality_violations 測試輸出：
{'items': [{'事業單位名稱': '大慶護理之家(許純毓)', '公告日期': '20200305', '裁處機關': '桃園市', '違反法條': '性別工作平等法第11條第1項', '違反法條內容': '懷孕解僱', '罰鍰金額': None, '資料列原始': {'主管機關': '桃園市', '公告日期': '20200305', '處分日期': '20200115', '處分字號': '109年府勞條字1090008089號', '事業單位名稱或負責人': '大慶護理之家(許純毓)', '違法法規法條': '性別工作平等法第11條第1項', '違反法規內容': '懷孕解僱', '罰鍰金額': '', '備註說明': '無'}}], 'count': 1, 'stats': {'count_by_year': {'2020': 1}}, 'source_url': 'https://apiservice.mol.gov.tw/OdService/download/A17000000J-030226-sop', 'fetched_at': '2025-09-24T14:02:08Z', 'meta': {'query': '大慶', 'since_year': 2020, 'partial_match': True}}
✅ ge_work_equality_violations output 格式正確


# Test MCP 

In [ ]:
import json
from mcp.server.fastmcp import FastMCP

# 假設你已經有 mcp instance
import server  

# 模擬一個 JSON-RPC 請求
req_list = {
    "jsonrpc": "2.0",
    "id": 1,
    "method": "tools/list",
    "params": {}
}

req_call = {
    "jsonrpc": "2.0",
    "id": 2,
    "method": "tools/call",
    "params": {
        "name": "esg_hr",
        "arguments": {
            "company": "茂生農經",
            "year": 113,
            "limit": 2
        }
    }
}

# FastMCP 有 dispatch 方法可以直接處理 JSON-RPC
resp_list = server.mcp.dispatch(req_list)
resp_call = server.mcp.dispatch(req_call)

print("tools/list 回應：")
print(json.dumps(resp_list, ensure_ascii=False, indent=2))

print("\ntools/call 回應：")
print(json.dumps(resp_call, ensure_ascii=False, indent=2))


In [ ]:
import asyncio
import json
import server  # 你已經有 mcp instance 在 server.py 裡

async def test_mcp():
    # 列出所有工具
    tools = await server.mcp.list_tools()
    print("tools/list 回應：")
    print(json.dumps(tools, ensure_ascii=False, indent=2))

    # 呼叫 esg_hr 工具
    result = await server.mcp.call_tool("esg_hr", {"company": "茂生農經", "year": 113, "limit": 2})
    print("\ntools/call 回應：")
    print(json.dumps(result, ensure_ascii=False, indent=2))

asyncio.run(test_mcp())


In [38]:
import json
import server  # 你已經有 mcp instance 在 server.py 裡

async def test_mcp():
    # 列出所有工具
    tools = await server.mcp.list_tools()
    print("tools/list 回應：", tools)
    


    # 呼叫 esg_hr 工具
    result = await server.mcp.call_tool("esg_hr", {"company": "茂生農經", "year": 113, "limit": 2})
    print("\ntools/call 回應：", result)
    

# 在 Jupyter notebook 裡用 await，而不是 asyncio.run()
await test_mcp()


tools/list 回應： [Tool(name='esg_hr', title=None, description='\n    查 ESG 人力發展（薪資/福利/女性主管比等）。會即時抓取 CSV，於工具內 ETL 後回傳。\n    Args:\n      company: 公司名稱（可含股份有限公司等尾綴）\n      year: 指定年度（可省略）\n      limit: 最多回傳筆數\n    Returns: dict(items=[...], source_url, fetched_at, meta)\n    ', inputSchema={'properties': {'company': {'title': 'Company', 'type': 'string'}, 'year': {'anyOf': [{'type': 'integer'}, {'type': 'null'}], 'default': None, 'title': 'Year'}, 'limit': {'default': 50, 'title': 'Limit', 'type': 'integer'}}, 'required': ['company'], 'title': 'esg_hrArguments', 'type': 'object'}, outputSchema=None, annotations=None, meta=None), Tool(name='labor_violations', title=None, description='\n    查勞動部違反勞基法紀錄（官方彙總）。\n    Args:\n      company: 事業單位名稱（可用部分關鍵字）\n      since_year: 公告日期的年份 >= since_year 才算\n      limit: 最多回傳筆數\n    ', inputSchema={'properties': {'company': {'title': 'Company', 'type': 'string'}, 'since_year': {'anyOf': [{'type': 'integer'}, {'type': 'null'}], 'default': None, 'title': '

## test all mcp tools

In [39]:
import json
import server  # mcp instance 在這裡

async def test_mcp():
    # 列出所有工具
    tools = await server.mcp.list_tools()
    print("tools/list 回應：", [t.name for t in tools])

    # 測試 esg_hr
    result1 = await server.mcp.call_tool("esg_hr", {
        "company": "茂生農經",
        "year": 113,
        "limit": 2
    })
    print("\n[esg_hr] 回應：", result1)

    # 測試 labor_violations
    result2 = await server.mcp.call_tool("labor_violations", {
        "company": "萊緹蛋糕坊",
        "since_year": 2025,
        "limit": 2
    })
    print("\n[labor_violations] 回應：", result2)

    # 測試 ge_work_equality_violations
    result3 = await server.mcp.call_tool("ge_work_equality_violations", {
        "company": "千才科技",
        "since_year": 2016,   # ⚡ 用真實有資料的年份
        "limit": 2
    })
    print("\n[ge_work_equality_violations] 回應：", result3)

# Jupyter 要用 await，不能 asyncio.run()
await test_mcp()


tools/list 回應： ['esg_hr', 'labor_violations', 'ge_work_equality_violations']

[esg_hr] 回應： [TextContent(type='text', text='{\n  "items": [\n    {\n      "公司代號": "1240",\n      "公司名稱": "茂生農經",\n      "年度": "113",\n      "員工薪資中位數": "649",\n      "員工薪資平均數": "841",\n      "女性主管比例": "6.00%",\n      "資料列原始": {\n        "\ufeff出表日期": "1140925",\n        "報告年度": "113",\n        "公司代號": "1240",\n        "公司名稱": "茂生農經",\n        "員工福利平均數(仟元/人)": "982",\n        "員工薪資平均數(仟元/人)": "841",\n        "非擔任主管職務之全時員工薪資平均數(仟元/人)": "740",\n        "非擔任主管之全時員工薪資中位數(仟元/人)": "649",\n        "管理職女性主管佔比": "6.00%"\n      }\n    }\n  ],\n  "count": 1,\n  "source_url": "https://mopsfin.twse.com.tw/opendata/t187ap46_O_5.csv",\n  "fetched_at": "2025-09-25T09:01:17Z",\n  "meta": {\n    "query": "茂生農經",\n    "year": 113,\n    "partial_match": false\n  }\n}', annotations=None, meta=None)]

[labor_violations] 回應： [TextContent(type='text', text='{\n  "items": [\n    {\n      "事業單位名稱": "賴慶鴻即萊緹蛋糕坊",\n      "公告日期": "20250829